In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sp

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import kruskal





# evaluate label propagation on the semi-supervised learning dataset

dataSet = sp.loadmat('breastw.mat') # appel du fichier breast.mat
X = dataSet['X']
y = dataSet['y']

Covariance=['tied', 'diag', 'full','spherical'] # les variables de covariance_type sont stockés dans une liste 
Params=['kmeans', 'random']# les variables de init_params sont stockés dans une liste 
skf = StratifiedKFold(n_splits=8) 

algo = 0
for i in Covariance: # boucle pour la covariance
        for j in Params:# boucle pour le parametre d'initialisation
                test=False 
                for train_index, test_index in skf.split(X, y):
                      
                
                        X_train, X_test = X[train_index,:], X[test_index,:] # on sépare le dataset en 2 pour le test et l'entrainement 
                        y_train, y_test = y[train_index], y[test_index]
                        
                        gmm = GaussianMixture(n_components=2,init_params=j,covariance_type=i) # appel de la fonction gaussian mixture model
                        gmm.fit(X_train,y_train)
                        y_pred=gmm.predict(X_test)
                        if precision_score(y_test,y_pred)< 0.05: # pour réaffecter les bons labels lorsqu'il y a un problème de labélisation
    
                               y_pred = np.array([0 if label == 1 else 1 for label in y_pred])
                        
                        F1=round(f1_score(y_test,y_pred),3) # récupère la valeur F1-score
                       
                       
                        if test==False :
                                data = np.array([[F1]]) # on stocke les F1 score dans un array
                                test=True
                        else :
                                data = np.append(data,np.array([F1]).reshape(1,1))
                if algo == 0  :
                        df=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)]) # on crée un dataframe à partir des F1
                        
                else :
                        df1=pd.DataFrame(data,columns=['[{},{}]'.format(i,j)])
                        df=pd.concat([df, df1], axis = 1) # on concatène les arrays pour former un tableau 
                algo = algo + 1
display(df) # on affiche le tabeau 
                

In [ ]:
import scikit_posthocs as sp
# test de kruskal pour comparer les variances 
t,p = kruskal(df['[spherical,kmeans]'],df['[spherical,random]'],df['[tied,kmeans]'],df['[tied,random]'],df['[diag,kmeans]'],df['[diag,random]'],df['[full,kmeans]'],df['[full,random]'])

# on affiche le t-test et la p-value 
print('t statistic: %.3f' % t)
print('p value: %.10f' % p)
# on réalise un post-hoc pour la comparaison 1 à 1
sp.posthoc_conover([df['[spherical,kmeans]'],df['[spherical,random]'],df['[tied,kmeans]'],df['[tied,random]'],df['[diag,kmeans]'],df['[diag,random]'],df['[full,kmeans]'],df['[full,random]']])